In [ ]:
import optuna
import pandas as pd
import numpy as np
from core.backtester import walk_forward
from core.helper import calc_cagr, calc_sharpe, calc_volatility, calc_sortino
import logging


def objective(trial, tickers,benchmark_ticker, start_invest , end_invest, start_capital):

  lookback_months = trial.suggest_int("lookback_months", 1, 12
  )
  skip_months = trial.suggest_int("skip_months", 0, 6)
  #top_k = trial.suggest_int("top_k", 1, 10)
  stop_loss = trial.suggest_float("stop_loss", 0,  0.5)


  try: 
    trade_records, capital, ben_capital, str_mon_returns, ben_mon_returns, \
        portfolio_value, benchmark_value, max_drawdown, max_drawdown_ben, \
          total_trades, wins, not_found, not_found_ben = walk_forward(
            tickers, benchmark_ticker, start_invest, end_invest, lookback_months, 
            skip_months, start_capital, stop_loss
        )
      
    sharpe = calc_sharpe(str_mon_returns, 0.04)
    cagr = calc_cagr(start_capital , capital, str_mon_returns, start_invest, end_invest)
    volatility = calc_volatility(str_mon_returns)
    sortino = calc_sortino(str_mon_returns, 0.04)

    win_rate = (wins / total_trades) if total_trades > 0 else 0

    score = sharpe*(1-max_drawdown) + (1+cagr/10)*(1+win_rate/2)

    if np.isnan(score) or np.isinf(score):
      return -1000
    
    return score

  except Exception as e:
    print(f"Error in trial:{e}")
    return -1000



In [9]:
tickers = "AAPL,MSFT,AMZN"
benchmark_ticker = "SPY"
start_invest = pd.to_datetime("2014-01-01")
end_invest = pd.to_datetime("2023-01-01")
start_capital = 10000

In [12]:
# while start_invest<=end_invest: 
#   start_tuning_period = pd.to_datetime(f"{start_invest.year - 4}-01-01")
#   end_tuning_period = pd.to_datetime(f"{start_invest.year}-01-01")

#   study = optuna.create_study(direction="maximize")
#   study.optimize(lambda trial: objective(trial, tickers, benchmark_ticker, start_tuning_period, end_tuning_period, start_capital), n_trials=10)

#   start_invest = (start_invest + pd.DateOffset(y=1)).replace(day=1)

# print(start_invest)

optuna.logging.set_verbosity(optuna.logging.INFO)

study = optuna.create_study(direction="maximize")
tickers = "AAPL,MSFT,AMZN"
benchmark_ticker = "SPY"
start_invest = pd.to_datetime("2022-01-01")
end_invest = pd.to_datetime("2023-01-01")
start_capital = 10000

study.optimize(lambda trial: objective(trial, tickers, benchmark_ticker, start_invest, end_invest, start_capital), n_trials=10)




[I 2025-10-11 15:54:24,968] A new study created in memory with name: no-name-888c5c65-89fe-42b9-a10b-fccc5dbf8dc2
[I 2025-10-11 15:54:25,085] Trial 0 finished with value: 3.255849190600033 and parameters: {'lookback_months': 19, 'skip_months': 4, 'stop_loss': 0.08985215546427466}. Best is trial 0 with value: 3.255849190600033.
[I 2025-10-11 15:54:25,134] Trial 1 finished with value: 2.9617890988741333 and parameters: {'lookback_months': 7, 'skip_months': 4, 'stop_loss': 0.03671854668382685}. Best is trial 0 with value: 3.255849190600033.
[I 2025-10-11 15:54:26,194] Trial 2 finished with value: 2.878920078103885 and parameters: {'lookback_months': 3, 'skip_months': 1, 'stop_loss': 0.10500375849349719}. Best is trial 0 with value: 3.255849190600033.
[I 2025-10-11 15:54:26,245] Trial 3 finished with value: 2.75262355746918 and parameters: {'lookback_months': 3, 'skip_months': 6, 'stop_loss': 0.46242761573811275}. Best is trial 0 with value: 3.255849190600033.
[I 2025-10-11 15:54:26,356] T

In [16]:
print(study.best_params)

start_tuning_period = pd.to_datetime(f"{start_invest.year - 4}-01-01")
end_tuning_period = pd.to_datetime(f"{start_invest.year}-01-01")

check = str(start_tuning_period)+str(end_tuning_period) 
print(check)

{'lookback_months': 17, 'skip_months': 6, 'stop_loss': 0.19178272057547052}
2018-01-01 00:00:002022-01-01 00:00:00


In [36]:
import pandas as pd
from core.helper import on_or_before_date, cal_start_end, get_stock_price, fetch_data
from optuna.pruners import MedianPruner

def walk_forward_with_tuning(tickers,benchmark_ticker, start_invest, end_invest, start_capital):
  
  max_lookback_months = 12
  max_skip_months = 6
  # Fetch the data
  df, not_found = fetch_data(tickers, start_invest, end_invest, max_lookback_months, max_skip_months)
  ben_data, not_found_ben = fetch_data(benchmark_ticker, start_invest, end_invest, max_lookback_months, max_skip_months)
  
  
  # Convert the start and end dates to datetime
  start_invest = pd.to_datetime(start_invest) 
  end_invest = pd.to_datetime(end_invest)
  #lookback_months= lookback_months
  #skip_months = skip_months
  top_k = 2
  trade_records= []
  capital=start_capital
  ben_capital = start_capital
  str_mon_returns = [] 
  ben_mon_returns= [] 
  portfolio_value = []
  benchmark_value =[] 
  peak_capital = start_capital
  max_drawdown = 0
  peak_capital_ben = start_capital
  max_drawdown_ben =0 
  total_trades=0
  wins= 0

  optimized_params = {} 

  while start_invest<end_invest: 

    current_year = start_invest.year

    # Check if the optimized params are already calculated
    if current_year not in optimized_params:
      
      start_tuning_period = pd.to_datetime(f"{start_invest.year - 4}-01-01")
      end_tuning_period = pd.to_datetime(f"{start_invest.year}-01-01")
      
      study = optuna.create_study(direction="maximize",
      sampler=optuna.samplers.TPESampler(n_startup_trials=10, multivariate=True),
      pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=3)
      )
      study.optimize(lambda trial: objective(trial, tickers, benchmark_ticker, start_tuning_period, end_tuning_period, start_capital), n_trials=50)

      optimized_params[current_year] = {
        "lookback_months": study.best_params["lookback_months"],
        "skip_months": study.best_params["skip_months"],
        "stop_loss": study.best_params["stop_loss"]
      }
    
    #Get the optimized params
    lookback_months = optimized_params[current_year]["lookback_months"]
    skip_months = optimized_params[current_year]["skip_months"]
    stop_loss = optimized_params[current_year]["stop_loss"]

    # Buy at the end of the month
    _, buy_date = on_or_before_date((start_invest + pd.offsets.MonthEnd(0)), df)

    if buy_date>end_invest:
      break

    # Sell at the end of the next month
    _, sell_date = on_or_before_date((buy_date + pd.offsets.MonthBegin(1) + pd.offsets.MonthEnd(0)), df)
    start_period, end_period = cal_start_end(buy_date, skip_months, lookback_months)
    
    # Start-End price and max diff
    start_price, start_date = on_or_before_date(start_period, df)
    end_price, end_date = on_or_before_date(end_period, df)
    diffs = (end_price-start_price).round(2)
    leaders = diffs.nlargest(top_k)
    
    # Equal weights
    w_each = 1/top_k
    capital_before = capital
    portfolio_return=0

    # Top-k leaders return
    for ticker in leaders.index:
      buy_price = get_stock_price(buy_date, ticker, df)
      sell_price = get_stock_price(sell_date, ticker, df)
      r = (sell_price-buy_price)/buy_price

      if r < (-stop_loss / 100):
        r = -stop_loss / 100
        sell_price = buy_price * (1 - stop_loss / 100)

      portfolio_return += w_each*r

      # Count trades and wins
      total_trades += 1
      if r >= 0:
        wins += 1

      trade_records.append({ 
        "buy_date": buy_date.date(),
        "sell_date": sell_date.date(),
        "window_start": start_date.date() ,
        "window_end" : end_date.date(),
        "ticker" : ticker,
        "start price": start_price[ticker],
        "end_price": end_price[ticker],
        "buy_price": buy_price,
        "sell_price":sell_price,
        "return":r 
      })
   
    #Update the capital
    capital *= (1+portfolio_return)

    
    # benchmark capital
    ben_capital_before = ben_capital
    ben_return = (ben_data.loc[sell_date,benchmark_ticker] - ben_data.loc[buy_date,benchmark_ticker])/ben_data.loc[buy_date,benchmark_ticker]
    ben_capital *= 1+ ben_return
    #ben_returns.append(round(ben_capital,2))  

    # Append monthly returns
    str_mon_returns.append(portfolio_return)
    ben_mon_returns.append(ben_return)

    # Summary return
    trade_records.append({
        "buy_date": "SUMMARY" ,
        "sell_date": "SUMMARY",
        "window_start": "SUMMARY" ,
        "window_end" : "SUMMARY",
        "ticker" : "SUMMARY",
        "start price": "SUMMARY",
        "end_price": "SUMMARY",
        "buy_price": "SUMMARY",
        "sell_price": "SUMMARY", 
        "return": portfolio_return ,
        "capital_before":round(capital_before,2),
        "capital_after": round(capital,2),
        "ben_capital_before": round(ben_capital_before,2),
        "ben_capital_after": round(ben_capital,2)
      })

    benchmark_value.append(ben_capital)
    portfolio_value.append(capital)

    # Max drawdown
    if capital > peak_capital:
      peak_capital = capital
    # Calculate current drawdown as percentage from peak
    current_drawdown = (peak_capital - capital) / peak_capital
    max_drawdown = max(max_drawdown, current_drawdown)

    # Drawdown benchmark
    if ben_capital>peak_capital_ben:
      peak_capital_ben=ben_capital
    current_dd_ben = (peak_capital_ben-ben_capital)/peak_capital_ben
    max_drawdown_ben  = max(max_drawdown_ben, current_dd_ben)

    #Date update for loop
    start_invest = (buy_date + pd.DateOffset(months=1)).replace(day=1)

  return trade_records, capital, ben_capital, str_mon_returns, ben_mon_returns, portfolio_value, benchmark_value, max_drawdown, max_drawdown_ben, total_trades, wins, not_found, not_found_ben




In [ ]:
# Test walk forward with tuning
tickers = "AAPL,MSFT,AMZN,GOOGL,META,NVDA,TSLA,UNH,JPM,V,JNJ,XOM,WMT,LLY,PG,MA,COST,HD,BAC"
benchmark_ticker = "SPY"
start_invest = pd.to_datetime("2018-01-01")
end_invest = pd.to_datetime("2023-01-01")
start_capital = 10000


trade_records, capital, ben_capital, str_mon_returns, ben_mon_returns, portfolio_value, benchmark_value, max_drawdown, max_drawdown_ben, total_trades, wins, not_found, not_found_ben = walk_forward_with_tuning(tickers, benchmark_ticker, start_invest, end_invest, start_capital)



In [ ]:
# Test walk forward with tuning
tickers = "AAPL,MSFT,AMZN,GOOGL,META,NVDA,TSLA,UNH,JPM,V,JNJ,XOM,WMT,LLY,PG,MA,COST,HD,BAC"
benchmark_ticker = "SPY"
start_invest = pd.to_datetime("2018-01-01")
end_invest = pd.to_datetime("2022-12-31")  # Full year
start_capital = 10000

print("=" * 80)
print("🧪 WALK FORWARD OPTIMIZATION COMPARISON TEST")
print("=" * 80)

# Test 1: With Optimization
print("\n🔧 TEST 1: Walk Forward WITH Optimization")
print("-" * 50)
trade_records_opt, capital_opt, ben_capital_opt, str_mon_returns_opt, ben_mon_returns_opt, portfolio_value_opt, benchmark_value_opt, max_drawdown_opt, max_drawdown_ben_opt, total_trades_opt, wins_opt, not_found_opt, not_found_ben_opt = walk_forward_with_tuning(tickers, benchmark_ticker, start_invest, end_invest, start_capital)

# Test 2: Without Optimization (Fixed Parameters)
print("\n📊 TEST 2: Walk Forward WITHOUT Optimization (Fixed Parameters)")
print("-" * 50)
trade_records_simple, capital_simple, ben_capital_simple, str_mon_returns_simple, ben_mon_returns_simple, portfolio_value_simple, benchmark_value_simple, max_drawdown_simple, max_drawdown_ben_simple, total_trades_simple, wins_simple, not_found_simple, not_found_ben_simple = walk_forward(tickers, benchmark_ticker, start_invest, end_invest, start_capital, 6, 1, 0.1)

# Results Comparison
print("\n📈 RESULTS COMPARISON")
print("=" * 80)

# Calculate metrics
def calculate_metrics(capital, start_capital, monthly_returns, max_drawdown, total_trades, wins):
    total_return = ((capital - start_capital) / start_capital) * 100
    cagr = calc_cagr(start_capital, capital, monthly_returns, start_invest, end_invest)
    sharpe = calc_sharpe(monthly_returns, 0.04)
    volatility = calc_volatility(monthly_returns)
    win_rate = (wins / total_trades) if total_trades > 0 else 0
    return total_return, cagr, sharpe, volatility, win_rate

# Optimized results
opt_total_return, opt_cagr, opt_sharpe, opt_volatility, opt_win_rate = calculate_metrics(
    capital_opt, start_capital, str_mon_returns_opt, max_drawdown_opt, total_trades_opt, wins_opt
)

# Simple results
simple_total_return, simple_cagr, simple_sharpe, simple_volatility, simple_win_rate = calculate_metrics(
    capital_simple, start_capital, str_mon_returns_simple, max_drawdown_simple, total_trades_simple, wins_simple
)

# Print comparison table
print(f"{'Metric':<20} {'Optimized':<15} {'Simple':<15} {'Difference':<15}")
print("-" * 65)
print(f"{'Total Return %':<20} {opt_total_return:<15.2f} {simple_total_return:<15.2f} {opt_total_return-simple_total_return:<15.2f}")
print(f"{'CAGR %':<20} {opt_cagr*100:<15.2f} {simple_cagr*100:<15.2f} {(opt_cagr-simple_cagr)*100:<15.2f}")
print(f"{'Sharpe Ratio':<20} {opt_sharpe:<15.2f} {simple_sharpe:<15.2f} {opt_sharpe-simple_sharpe:<15.2f}")
print(f"{'Volatility %':<20} {opt_volatility*100:<15.2f} {simple_volatility*100:<15.2f} {(opt_volatility-simple_volatility)*100:<15.2f}")
print(f"{'Max Drawdown %':<20} {max_drawdown_opt*100:<15.2f} {max_drawdown_simple*100:<15.2f} {(max_drawdown_opt-max_drawdown_simple)*100:<15.2f}")
print(f"{'Win Rate %':<20} {opt_win_rate*100:<15.2f} {simple_win_rate*100:<15.2f} {(opt_win_rate-simple_win_rate)*100:<15.2f}")
print(f"{'Total Trades':<20} {total_trades_opt:<15} {total_trades_simple:<15} {total_trades_opt-total_trades_simple:<15}")
print(f"{'Final Capital':<20} {capital_opt:<15.2f} {capital_simple:<15.2f} {capital_opt-capital_simple:<15.2f}")

print("\n🎯 CONCLUSION")
print("=" * 80)
if capital_opt > capital_simple:
    improvement = ((capital_opt - capital_simple) / capital_simple) * 100
    print(f"✅ Optimization IMPROVED performance by {improvement:.2f}%")
else:
    decline = ((capital_simple - capital_opt) / capital_simple) * 100
    print(f"❌ Optimization DECREASED performance by {decline:.2f}%")